In [1]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [24]:
from transformers import BertTokenizer, BertModel
import torch
from datasets import load_dataset
from tqdm.notebook import tqdm
import numpy as np
import os

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [27]:
def get_datasets():
  #imdb = load_dataset('imdb')
  #sst2 = load_dataset('sst2')
  yelp_review_full = load_dataset('yelp_review_full')
  return {'yelp': yelp_review_full} #{'imdb': imdb, 'sst2': sst2, 'yelp': yelp_review_full}

datasets_dict = get_datasets()

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
model = model.to(device)

In [31]:
def get_pt_embeddings(datasets_dict, perc_split = 0.1):

  if not os.path.exists(f'/content/datasets'): os.mkdir(f'/content/datasets')

  for ds_name, dataset in datasets_dict.items():

    if not os.path.exists(f'/content/datasets/{ds_name}'): os.mkdir(f'/content/datasets/{ds_name}')

    for split in ['train', 'test']:

      print(f'\n------------ {ds_name} - {split} ------------')

      if not os.path.exists(f'/content/datasets/{ds_name}/{split}'): os.mkdir(f'/content/datasets/{ds_name}/{split}')

      len_train = len(dataset[split])
      dim_split = int(len_train * perc_split)

      range_splits = [(idx, len_train) if idx + dim_split > len_train else (idx, idx + dim_split) for idx in range(0, len_train, dim_split)]

      for idx, (strat_range, end_range) in enumerate(range_splits):

        embeddings_tensor = torch.empty((0,768)).to(device)
        outputs = torch.empty((0,768)).to(device)

        torch.save(embeddings_tensor, f'/content/datasets/{ds_name}/{split}/embeddings_{idx}.pt')

        print(f'---------- Split {idx} ----------')

        ds_dict = dict(dataset[split][strat_range:end_range].items())

        if 'text' in ds_dict: ds_dict = ds_dict['text']
        elif 'sentence' in ds_dict: ds_dict = ds_dict['sentence']
        else: raise Exception('Invalid key for datasets')


        for text in tqdm(ds_dict, total = len(ds_dict), leave=False):

          embeddings_tensor = torch.load(f'/content/datasets/{ds_name}/{split}/embeddings_{idx}.pt')

          encoded_text = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)

          outputs = model(**encoded_text, output_hidden_states=True).last_hidden_state[:, 0, :]

          embeddings_tensor = torch.cat((embeddings_tensor, outputs), dim=0)

          torch.save(embeddings_tensor, f'/content/datasets/{ds_name}/{split}/embeddings_{idx}.pt')


In [35]:
def get_npy_embeddings(datasets_dict):
  for ds_name in list(datasets_dict.keys()):
    for split in ['train', 'test']:

      print(f'\n------------ {ds_name} - {split} ------------')

      to_array = np.empty((0,768))
      for idx in tqdm(range(10), total = len(range(10)), leave=False):
        to_array = np.vstack((to_array,
                              torch.load(f'{ds_name}/{split}/embeddings_{idx}.pt').cpu().detach().numpy()))
      np.save(f'{ds_name}/{split}/embeddings.npy', to_array)

In [ ]:
get_pt_embeddings(datasets_dict, 0.05)
get_npy_embeddings(datasets_dict)